# Statistics
By the end of this lecture you will be able to:
- calculate statistics on a `DataFrame` or expression
- calculate cumulative, rolling and exponentially-weighted statistics
- do row-wise calculations

In [1]:
import polars as pl

In [2]:
csv_file = "../data/titanic.csv"

In [15]:
df = pl.read_csv(csv_file)
df.head(3)

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
i64,i64,i64,str,str,f64,i64,i64,str,f64,str,str
1,0,3,"""Braund, Mr. Ow…","""male""",22.0,1,0,"""A/5 21171""",7.25,null,"""S"""
2,1,1,"""Cumings, Mrs. …","""female""",38.0,1,0,"""PC 17599""",71.2833,"""C85""","""C"""
3,1,3,"""Heikkinen, Mis…","""female""",26.0,0,0,"""STON/O2. 31012…",7.925,null,"""S"""


## Statistics on a `DataFrame`

We can call statistical methods on all columns of a `DataFrame` such as `mean`,`min`,`max` etc

In [4]:
df.mean()

PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
f64,f64,f64,str,str,f64,f64,f64,str,f64,str,str
446.0,0.383838,2.308642,null,null,29.699118,0.523008,0.381594,null,32.204208,null,null


We can get an overview of the statistics of a `DataFrame` with `describe`

In [5]:
df.describe()

statistic,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
str,f64,f64,f64,str,str,f64,f64,f64,str,f64,str,str
"""count""",891.0,891.0,891.0,"""891""","""891""",714.0,891.0,891.0,"""891""",891.0,"""204""","""889"""
"""null_count""",0.0,0.0,0.0,"""0""","""0""",177.0,0.0,0.0,"""0""",0.0,"""687""","""2"""
"""mean""",446.0,0.383838,2.308642,null,null,29.699118,0.523008,0.381594,null,32.204208,null,null
"""std""",257.353842,0.486592,0.836071,null,null,14.526497,1.102743,0.806057,null,49.693429,null,null
"""min""",1.0,0.0,1.0,"""Abbing, Mr. An…","""female""",0.42,0.0,0.0,"""110152""",0.0,"""A10""","""C"""
"""25%""",224.0,0.0,2.0,null,null,20.0,0.0,0.0,null,7.925,null,null
"""50%""",446.0,0.0,3.0,null,null,28.0,0.0,0.0,null,14.4542,null,null
"""75%""",669.0,1.0,3.0,null,null,38.0,1.0,0.0,null,31.0,null,null
"""max""",891.0,1.0,3.0,"""van Melkebeke,…","""male""",80.0,8.0,6.0,"""WE/P 5735""",512.3292,"""T""","""S"""


Note that for string columns the values are cast to string.

We can specify the percentiles used in `describe` by passing a `list` or `tuple` of decimals

In [6]:
df.describe(percentiles=(0.1,0.3,0.5,0.7,0.9))

statistic,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
str,f64,f64,f64,str,str,f64,f64,f64,str,f64,str,str
"""count""",891.0,891.0,891.0,"""891""","""891""",714.0,891.0,891.0,"""891""",891.0,"""204""","""889"""
"""null_count""",0.0,0.0,0.0,"""0""","""0""",177.0,0.0,0.0,"""0""",0.0,"""687""","""2"""
"""mean""",446.0,0.383838,2.308642,null,null,29.699118,0.523008,0.381594,null,32.204208,null,null
"""std""",257.353842,0.486592,0.836071,null,null,14.526497,1.102743,0.806057,null,49.693429,null,null
"""min""",1.0,0.0,1.0,"""Abbing, Mr. An…","""female""",0.42,0.0,0.0,"""110152""",0.0,"""A10""","""C"""
"""10%""",90.0,0.0,1.0,null,null,14.0,0.0,0.0,null,7.55,null,null
"""30%""",268.0,0.0,2.0,null,null,22.0,0.0,0.0,null,8.05,null,null
"""50%""",446.0,0.0,3.0,null,null,28.0,0.0,0.0,null,14.4542,null,null
"""70%""",624.0,1.0,3.0,null,null,36.0,1.0,0.0,null,27.0,null,null


## Statistics in an expression
We can calculate statistics in an expression

In [7]:
(
    df
    .select(
        pl.col('Fare').mean()
    )
)

Fare
f64
32.204208


The statistics available include:
- count
- sum
- product
- min
- median
- mean
- max
- std (standard deviation)
- var (variance)
- skew
- kurtosis
- entropy

## Rolling statistics
We can calculate rolling statistics in an expression.

> For rolling time series analysis see the rolling lecture in the time series section of the course.

We first create a simple `DataFrame` with sequential values

In [23]:
df_rolling = (
    pl.DataFrame(
        {
            "value":range(12),
        }
    )
)
df_rolling.head()

value
i64
0
1
2
3
4


We take the rolling mean over 4 values by setting the `window_size` to be 4

In [24]:
(
    df_rolling
    .with_columns(
        rolling_mean_value = pl.col("value").rolling_mean(window_size=4)
    )
    .head(5)
)

value,rolling_mean_value
i64,f64
0,null
1,null
2,null
3,1.5
4,2.5


Note that by default the first non-`null` value is on the 4th row.

We can calculate the statistic with fewer values than the `window_size` by setting the `min_periods` argument

In [25]:
(
    df_rolling
    .with_columns(
        rolling_mean_value = pl.col("value").rolling_mean(window_size=4),
        rolling_mean_value_min_periods = pl.col("value").rolling_mean(window_size=4,min_periods=1)

    )
).head()

value,rolling_mean_value,rolling_mean_value_min_periods
i64,f64,f64
0,null,0.0
1,null,0.5
2,null,1.0
3,1.5,1.5
4,2.5,2.5


In the examples above the statistics are *backward-looking*. That is, the value on the 4th row is the average of the first four rows. We can instead center the statistic with the `center` argument (note that we use a window size of 5 here)

In [26]:
(
    df_rolling
    .with_columns(
        rolling_mean_value = pl.col("value").rolling_mean(window_size=5),
        rolling_mean_value_center = pl.col("value").rolling_mean(window_size=5,center=True)
    ).head(5)
)

value,rolling_mean_value,rolling_mean_value_center
i64,f64,f64
0,null,null
1,null,null
2,null,2.0
3,null,3.0
4,2.0,4.0


In this case the values on the third row are the mean of the first five rows.

See the full range of rolling statistics here: https://pola-rs.github.io/polars/py-polars/html/reference/expressions/computation.html

## Exponentially-weighted statistics
Polars has exponentially-weighted statistics available as expressions.

The `span` parameter determines the "alpha" value used in the exponential weighting formula, which is given by:

alpha = 2 / (L + 1)

where L is the span value. The alpha value determines the rate of decay of the weights applied to each data point in the calculation. A higher alpha (or lower span) means that more weight is given to recent data points, while a lower alpha (or higher span) value means that more weight is given to older data points.

In [27]:
(
    df_rolling
    .with_columns(
        ewm_mean_value = pl.col("value").ewm_mean(span=4)
    ).head(5)
)

value,ewm_mean_value
i64,f64
0,0.0
1,0.625
2,1.326531
3,2.095588
4,2.921582


For the `ewm_mean` the `min_periods` is 1 by default.

Exponentially-weighted statistics available are:
- `ewm_mean`
- `ewm_std`
- `ewm_var`

### Multiple statistics
We can use `prefix` or `suffix` when calculating multiple statistics on the same column or columns to avoid name collisions

In [30]:
(
    df_rolling
    .select(
        pl.col(pl.Int64).min().name.suffix("_min"),
        pl.col(pl.Int64).max().name.suffix("_max"),
    )
)

value_min,value_max
i64,i64
0,11


We can also do arithmetic with statistics. 

In this example we calculate a min-max scaler

In [33]:
(
    df
    .with_columns(
        ((pl.col(pl.Int64) - pl.col(pl.Int64).min()) / (pl.col(pl.Int64).max() - pl.col(pl.Int64).min())).name.suffix("_scaled")
    )
)

value,value_scaled
i64,f64
0,0.0
1,0.090909
2,0.181818
3,0.272727
4,0.363636
5,0.454545
6,0.545455
7,0.636364
8,0.727273


I have found that doing this calculation in Polars can be faster than in numpy: https://www.rhosignal.com/posts/polars-minmax/

## Horizontal computations
To illustrate horizontal computations we define a simple `DataFrame` with two columns

In [34]:
df_hor = pl.DataFrame(
    {
        "vals1":[0,1,2],
        "val2":[3,4,5]
    }
)
df_hor

vals1,val2
i64,i64
0,3
1,4
2,5


Polars has a few dedicated horizontal aggregation functions (with hopefully more to come in the future). The output of these expressions is the name of the first column that goes into them so we need an `alias` to avoid overwriting an existing column with the output statistic

In [36]:
(
    df_hor
    .with_columns(
        pl.max_horizontal(pl.all()).alias("max"),
        pl.min_horizontal(pl.all()).alias("min"),
        pl.sum_horizontal(pl.all()).alias("sum"),
        
    )
)

vals1,val2,max,min,sum
i64,i64,i64,i64,i64
0,3,3,0,3
1,4,4,1,5
2,5,5,2,7


There is also a horizontal `cum_sum`. As any `cum_sum` is not an aggregation (i.e. the output is not a scalar but a `Series` the same length as the input) the `cum_sum_horizontal` output is a `pl.Struct` column with the number of fields equal to the number of columns

In [37]:
(
    df_hor
    .with_columns(
        pl.cum_sum_horizontal(pl.all()),
        
    )
)

vals1,val2,cum_sum
i64,i64,struct[2]
0,3,"{0,3}"
1,4,"{1,5}"
2,5,"{2,7}"


To compute bespoke horizontal computations we can concatenate the values on each row into a `pl.List` column. Recall that each row in a `pl.List` column is a `Series` so we can then use expressions on each row.

In this example we want to calculate the horizontal mean. First we concatenate the values into a list

In [38]:
(
    df_hor
    .with_columns(
        pl.concat_list(pl.all()).alias("concat")
    )
)

vals1,val2,concat
i64,i64,list[i64]
0,3,"[0, 3]"
1,4,"[1, 4]"
2,5,"[2, 5]"


To do our horizontal calculation we then use the approaches we saw in the lectures on the list columns:
- using `list.` expressions
- calling `explode` on the list column
- using `list.eval`

# Exercises

In the exercises you will develop your understanding of:
- calculating statistics on a column
- calculating statistics on multiple columns of the same dtype
- calculating cumulative statistics
- doing horizontal calculations

### Exercise 1 - calculating multiple statistics
Calculate the mean and median of the `Age` column for passengers in 1st class

In [ ]:
(
    pl.read_csv(csv_file)
    <blank>
)

Add a new column called `Age_delta` that is the difference between the age and the average age of all passengers

In [ ]:
(
    pl.read_csv(csv_file)
    .with_columns(
        <blank>
    )
    .select(
        'Age','Age_delta'
    )
    .head(10)
)

Add another column called `Age_z` that has the z-score for the `Age` where the z-score is the (age - average age of the column) divided by the standard deviation of the age column

Create these new columns for all floating point columns in the CSV. Add a `pipe` command if you want to sort the columns alphabetically

### Exercise 2
We have the following `DataFrame` with values that occur in sequences in the `records` column

In [ ]:
records = (
    pl.DataFrame(
        {
            "values":['A','A','A','B','B','A','A']
        }
    )
)
records

We want to identify sequences of rows with the same values in the `values` column to get the following output

In [ ]:
(
    pl.DataFrame(
        {
            "values":['A','A','A','B','B','A','A'],
            "groups":[0,0,0,1,1,2,2]
        }
    )
)

Try this yourself or follow the step-by-step guide below if you need help. 

Note that one way to do this involves the `shift` expression that we haven't met before. This moves all values in a column by the specified number of places

In [ ]:
(
    records
    .with_columns(
        pl.col("values").shift(1).alias("shifted")
    )
)

Step-by-step approach:

Check if the value in `values` in each row is **not** equal to the value in the previous row. Do this in a boolean column called `notEqualsPrevious`

Use a cumulative function on `notEqualsPrevious` to increment an integer value whenever a row that is not equal to the previous value is encountered. If you have any strange results you may need to cast Boolean values to integers first!

### Exercise 3
We are given the following data from three weather stations over 8 months

In [ ]:
data = [
    {"Year": 2023, "Month": "Jan", "Station_A (°C)": 20.5, "Station_B (°C)": 18.0, "Station_C (°C)": 25.0},
    {"Year": 2023, "Month": "Feb", "Station_A (°C)": 21.0, "Station_B (°C)": 18.5, "Station_C (°C)": 26.0},
    {"Year": 2023, "Month": "Mar", "Station_A (°C)": 23.5, "Station_B (°C)": 20.0, "Station_C (°C)": 28.0},
    {"Year": 2023, "Month": "Apr", "Station_A (°C)": 25.0, "Station_B (°C)": 22.0, "Station_C (°C)": 29.5},
    {"Year": 2023, "Month": "May", "Station_A (°C)": 26.5, "Station_B (°C)": 23.0, "Station_C (°C)": 30.0},
    {"Year": 2023, "Month": "Jun", "Station_A (°C)": 28.0, "Station_B (°C)": 24.0, "Station_C (°C)": 32.0},
    {"Year": 2023, "Month": "Jul", "Station_A (°C)": 29.0, "Station_B (°C)": 25.5, "Station_C (°C)": 33.5},
    {"Year": 2023, "Month": "Aug", "Station_A (°C)": 30.0, "Station_B (°C)": 26.0, "Station_C (°C)": 34.0}
]
df_weather = pl.DataFrame(data)
df_weather

Add a column with the max temperature for each month

In [ ]:
(
    df_weather
    .with_columns(
        <blank>
    )
)

Add another column called `std` with the standard deviation of measurements for each month rounded off to one decimal place

# Solutions

### Solution to Exercise 1 
Calculate the mean and median of the `Age` column for passengers in 1st class

In [ ]:
(
    pl.read_csv(csv_file)
    .filter(
        pl.col('Pclass') == 1
    )
    .select(
        [
            pl.col('Age').mean().alias('Age_mean'),
            pl.col('Age').median().alias('Age_median')
        ]
    )
)

Add a new column called `Age_delta` that is the difference between the age and the average age of all passengers

In [ ]:
(
    pl.read_csv(csv_file)
    .with_columns(
        (pl.col('Age') - pl.col('Age').mean()).alias('Age_delta')
    )
    .select(
        'Age','Age_delta'
    )
    .head(10)
)

Add a further column called `Age_z` that has the z-score for the `Age`: this is the (age - average age of the column) divided by the standard deviation of the age column

In [ ]:
(
    pl.read_csv(csv_file)
    .with_columns(
        [
            (pl.col('Age') - pl.col('Age').mean()).alias('Age_delta'),
            ((pl.col('Age') - pl.col('Age').mean())/pl.col('Age').std()).alias('Age_z')
        ]
    )
    .select(
        'Age','Age_delta','Age_z'
    )
    .head(10)
)

Create these new columns for all floating point columns in the CSV. Add a `pipe` command if you want to sort the columns alphabetically

In [ ]:
(
    pl.read_csv(csv_file)
    .with_columns(
        [
            (pl.col(pl.Float64) - pl.col(pl.Float64).mean()).name.suffix('_delta'),
            ((pl.col(pl.Float64) - pl.col(pl.Float64).mean())/pl.col(pl.Float64).std()).name.suffix('_z')
        ]
    )
    .select(
        pl.col(pl.Float64)
    )
    .pipe(lambda df:df.select(sorted(df.columns)))
    .head(10)
)

### Solution to exercise 2
We have the following `DataFrame` with values that occur in sequences in the `records` column

In [ ]:
records = (
    pl.DataFrame(
        {
            "values":['A','A','A','B','B','A','A']
        }
    )
)
records

We want to identify groups of rows with the same consecutive values to get the following output. The column `groups` shows how long the sequence which that row belongs to it.

In [ ]:
(
    pl.DataFrame(
        {
            "values":['A','A','A','B','B','A','A'],
            "groups":[0,0,0,1,1,2,2]
        }
    )
)

Check if the value in `values` in each row is **not** equal to the value in the previous row. Do this in a boolean column called `notEqualsPrevious`

In [ ]:
(
    records
    .with_columns(
        (pl.col('values') != pl.col('values').shift(1)).alias('notEqualsPrevious')
    )
)

Use a cumulative function on `notEqualsPrevious` to increment an integer value whenever a row that is not equal to the previous value is encountered. If you have any strange results you may need to cast Boolean values to integers first!

In [ ]:
(
    records
    .with_columns(
        (pl.col('values') != pl.col('values').shift(1)).alias('notEqualsPrevious')
    )
    .with_columns(
        (pl.col('notEqualsPrevious').cast(pl.Int32).cum_sum().fill_null(0)).alias('groups')
    )
)

### Solution to exercise 3
We are given the following data from three weather stations over 8 months

In [ ]:
data = [
    {"Year": 2023, "Month": "Jan", "Station_A (°C)": 20.5, "Station_B (°C)": 18.0, "Station_C (°C)": 25.0},
    {"Year": 2023, "Month": "Feb", "Station_A (°C)": 21.0, "Station_B (°C)": 18.5, "Station_C (°C)": 26.0},
    {"Year": 2023, "Month": "Mar", "Station_A (°C)": 23.5, "Station_B (°C)": 20.0, "Station_C (°C)": 28.0},
    {"Year": 2023, "Month": "Apr", "Station_A (°C)": 25.0, "Station_B (°C)": 22.0, "Station_C (°C)": 29.5},
    {"Year": 2023, "Month": "May", "Station_A (°C)": 26.5, "Station_B (°C)": 23.0, "Station_C (°C)": 30.0},
    {"Year": 2023, "Month": "Jun", "Station_A (°C)": 28.0, "Station_B (°C)": 24.0, "Station_C (°C)": 32.0},
    {"Year": 2023, "Month": "Jul", "Station_A (°C)": 29.0, "Station_B (°C)": 25.5, "Station_C (°C)": 33.5},
    {"Year": 2023, "Month": "Aug", "Station_A (°C)": 30.0, "Station_B (°C)": 26.0, "Station_C (°C)": 34.0}
]
df_weather = pl.DataFrame(data)
df_weather

Add a column with the max temperature for each month

In [ ]:
(
    df_weather
    .with_columns(
        pl.max_horizontal(pl.col(pl.Float64)).alias("max"),
    )
)

Add another column called `std` with the standard deviation of measurements for each month rounded off to one decimal place

In [ ]:
(
    df_weather
    .with_columns(
        pl.max_horizontal(pl.col(pl.Float64)).alias("max"),
        pl.concat_list(pl.col(pl.Float64))
        .list.eval(
            pl.element().std()
        )
        .list.get(0)
        .round(1)
        .alias("std")
    )
)